# 07 - SMPC

## Materials

### `Gevent`

Two SMPC actions which require results from another machine on the network that takes half a second to arrive.

In [1]:
import gevent
import time
start = time.time()
end = None
def smpc_action_1():
    print('Starting SMPC Action 1')
    # waiting for some network result will block
    time.sleep(0.5)
    print('Finished SMPC Action 1')

def smpc_action_2():
    global end
    print('Starting SMPC Action 2')
    # data here no need to wait
    time.sleep(0)
    print('Finished SMPC Action 2')
    end = time.time()

_ = gevent.joinall([
    gevent.spawn(smpc_action_1),
    gevent.spawn(smpc_action_2),
])
print(f"\nTime to finish SMPC Action 2:", round(end - start, 2))

Starting SMPC Action 1
Finished SMPC Action 1
Starting SMPC Action 2
Finished SMPC Action 2

Time to finish SMPC Action 2: 0.5


We had to wait 0.5s for the `smpc_action_1` to complete. Instead if we use `gevent` we can run both actions in parallel.

In [2]:
import gevent
start = time.time()
end = None
def smpc_action_1():
    print('Starting SMPC Action 1')
    # waiting for a network result doesn't matter
    gevent.sleep(0.5)
    print('Finished SMPC Action 1')

def smpc_action_2():
    global end
    print('Starting SMPC Action 2')
    # waiting for a network result doesn't matter
    gevent.sleep(0)
    print('Finished SMPC Action 2')
    end = time.time()

_ = gevent.joinall([
    gevent.spawn(smpc_action_1),
    gevent.spawn(smpc_action_2),
])
print(f"\nTime to finish SMPC Action 2:", round(end - start, 2))

Starting SMPC Action 1
Starting SMPC Action 2
Finished SMPC Action 2
Finished SMPC Action 1

Time to finish SMPC Action 2: 0.0


Since `SMPC Action 2` was ready to go, we can see that it was able to start nearly immediately and complete all while the first one was waiting on the network. This is because the `SMPC Action 1` is telling `gevent`, i'm waiting so I `yield` my CPU time to someone else.

### SMPC in action

In [1]:
import syft as sy
sy.requires("==0.7")
import numpy as np

a = sy.Tensor(np.array([1, 2, 3])) # Alice's data  
b = sy.Tensor(np.array([4, 5, 6])) # Bob's data
c = a + b
d = a * b
local_c = c
local_d = d

✅ The installed version of syft==0.7.0 matches the requirement ==0.7


In [3]:
print(type(local_c))
local_c

<class 'syft.core.tensor.tensor.Tensor'>


Tensor(child=[5 7 9])

### 2 domains

Start the first `domain` named `alice`

In [ ]:
!hagrid launch alice domain to docker:8081 --tag=0.7.0 --tail

In [67]:
import hagrid
hagrid.check("localhost:8081")

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8081/login               │ ✅ │
│ api       │ http://localhost:8081/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

Start the second `domain` named `bob`

In [ ]:
!hagrid launch bob domain to docker:8082 --tag=0.7.0 --tail

In [66]:
hagrid.check("localhost:8082")

┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━┓
┃ PyGrid    ┃ Info                                      ┃    ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━┩
│ UI (βeta) │ http://localhost:8082/login               │ ✅ │
│ api       │ http://localhost:8082/api/v1/openapi.json │ ✅ │
└───────────┴───────────────────────────────────────────┴────┘

Login to the `domains`

In [8]:
alice_domain = sy.login(email="info@openmined.org", 
                        password="changethis", 
                        port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into alice... done!


In [10]:
bob_domain = sy.login(email="info@openmined.org", 
                      password="changethis", 
                      port=8082)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into bob... done!


Upload some data to the `domains`

In [6]:
a, b

(Tensor(child=[1 2 3]), Tensor(child=[4 5 6]))

In [11]:
alice_domain.load_dataset(
    assets={"data": a},
    name="alice_dataset",
    description="Alice's Private Data",
    skip_checks=True
)

Loading dataset... checking asset types...                                                                                                                                    

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/dk


Loading dataset... uploading...🚀                        

Uploading `data`: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00,  4.43it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


In [12]:
bob_domain.load_dataset(
    assets={"data": b},
    name="bob_dataset",
    description="Bob's Private Data",
    skip_checks=True
)

Loading dataset... uploading...🚀                                                                                                                                             

Uploading `data`: 100%|███████████████████████████████████████████████| 1/1 [00:00<00:00,  4.17it/s]

Dataset is uploaded successfully !!! 🎉

Run `<your client variable>.datasets` to see your new dataset loaded into your machine!


Create `Data Scientist` Account

In [14]:
starting_budget = starting_budget = 999999
alice_domain.create_user(
    name = "Alice Amidala",
    email = "alice@naboo.net",
    password = "gungan",
    budget = starting_budget,
)

User created successfully!


{'name': 'Alice Amidala',
 'email': 'alice@naboo.net',
 'password': 'gungan',
 'url': 'localhost'}

In [15]:
bob_domain.create_user(
    name = "Bob Afett",
    email = "bob@hutt.pizza",
    password = "hansolo",
    budget = starting_budget,
)

User created successfully!


{'name': 'Bob Afett',
 'email': 'bob@hutt.pizza',
 'password': 'hansolo',
 'url': 'localhost'}

Login into the `Data Scientist` accounts to get a different `domain` handles with names `alice_ds` and `bob_ds`

In [16]:
alice_ds = sy.login(port=8081, 
                    email="alice@naboo.net", 
                    password="gungan")

Connecting to localhost... done! 	 Logging into alice... done!


In [17]:
bob_ds = sy.login(port=8082, 
                  email="bob@hutt.pizza", 
                  password="hansolo")

Connecting to localhost... done! 	 Logging into bob... done!


Get the pointers to the private data uploaded by `Alice` and `Bob`

In [20]:
alice_dataset_ptr = alice_ds.datasets[-1]
alice_dataset_ptr

Dataset: alice_dataset
Description: Alice's Private Data



Asset Key,Type,Shape
"[""data""]",Tensor,"(3,)"


In [21]:
alice_ptr = alice_dataset_ptr["data"]
alice_ptr

<TensorPointer -> alice:c0bfd3f19a974bbdb73db6ec0296f5e6, status=Ready>

In [23]:
bob_dataset_ptr = bob_ds.datasets[-1]
bob_dataset_ptr

Dataset: bob_dataset
Description: Bob's Private Data



Asset Key,Type,Shape
"[""data""]",Tensor,"(3,)"


In [27]:
bob_ptr = bob_dataset_ptr["data"]
bob_ptr

<TensorPointer -> bob:b05cbdf090c7462f8f646b136938131d, status=Ready>

In [28]:
a = alice_ptr
b = bob_ptr
c = a + b
d = a * b

As `a` and `b` are on different domains, `syft` automatically creates `SMPC Tensors` and handles all the logic to do `secret-sharing` transparently for us because it knows the `Pointers` are on different `domains`.

In [37]:
c



MPCTensor.shape=(3,)
	 .child[0] = <TensorPointer -> alice:dbe5fdaa9f9cf050ef770e5e21049e85, status=Processing>
	 .child[1] = <TensorPointer -> bob:dbe5fdaa9f9cf050ef770e5e21049e85, status=Processing>

In [38]:
d



MPCTensor.shape=(3,)
	 .child[0] = <TensorPointer -> alice:d710de2f1c0386078cc04a5464f275dc, status=Processing>
	 .child[1] = <TensorPointer -> bob:d710de2f1c0386078cc04a5464f275dc, status=Processing>

Let's try to get the value of `c` and see what happens

In [55]:
# ⚔️ Runnable Code
try:
    c.get(delete_obj=False)
except Exception:
    print("You do not have permission to .get() Object on the node. Please submit a request.")

Unable to json decode message. Expecting value: line 1 column 1 (char 0)
You do not have permission to .get() Object on the node. Please submit a request.


### Requests

#### Data requests

In [76]:
c.request(reason="These are the Droids youre looking for! 👋")

In [77]:
alice_domain.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Alice Amidala,alice@naboo.net,Data Scientist,BUDGET,pending,I would like additional privacy budget to perf...,<UID: 140374721730480dba492ceed19dfbe7>,None,[],11.0,999999.0


In [69]:
bob_domain.requests

""


#### Privacy Budget Request

In [70]:
alice_ds.privacy_budget

999999.0

In [71]:
alice_ds.request_budget(eps=11, 
                        reason="I would like additional privacy budget to perfom computation")

Requested 11 epsilon of budget. Call .privacy_budget to see if your budget has arrived!


### Response

In [75]:
alice_domain.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Alice Amidala,alice@naboo.net,Data Scientist,BUDGET,pending,I would like additional privacy budget to perf...,<UID: 140374721730480dba492ceed19dfbe7>,None,[],11.0,999999.0


In [78]:
alice_domain.requests[0].accept()

In [79]:
alice_domain.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Alice Amidala,alice@naboo.net,Data Scientist,BUDGET,accepted,I would like additional privacy budget to perf...,<UID: 140374721730480dba492ceed19dfbe7>,None,[],11.0,999999.0


In [80]:
alice_ds.privacy_budget

1000010.0

In [81]:
alice_domain.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Alice Amidala,alice@naboo.net,Data Scientist,BUDGET,accepted,I would like additional privacy budget to perf...,<UID: 140374721730480dba492ceed19dfbe7>,None,[],11.0,999999.0


In [ ]:
smpc_c = c.get(delete_obj=False)

In [83]:
bob_domain.requests

""
